# **Import**

In [1]:
import tarfile
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import yaml
import os, torch
from pathlib import Path
from torchvision.ops import box_iou
import warnings
import sys, time, warnings, subprocess
from kaggle_secrets import UserSecretsClient
warnings.filterwarnings("ignore")

# **Globals**

In [2]:
# define the alphabet used for plate decoding. each LP number is comprised of a Chinese character, a letter, and five letters or numbers.

PROVINCES = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑",
             "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤",
             "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁",
             "新", "警", "学", "O"]

ALPHA = ['A','B','C','D','E','F','G','H','J','K',
             'L','M','N','P','Q','R','S','T','U','V',
             'W','X','Y','Z','O'] 

ADS = ['A','B','C','D','E','F','G','H','J','K',
       'L','M','N','P','Q','R','S','T','U','V',
       'W','X','Y','Z','0','1','2','3','4','5',
       '6','7','8','9','O']

SRC_IMG_DIR = "/kaggle/working/ccpd_subset_base/train"
OUT_BASE = "/kaggle/working/ccpd_yolo_dataset"
IMG_W, IMG_H = 720, 1160
CLASS_ID = 0

# specify the path for train & val for yolov5
CONTENT = {
    'train': '/kaggle/working/ccpd_yolo_dataset/images/train',
    'val': '/kaggle/working/ccpd_yolo_dataset/images/val',
    'nc': 1,
    'names': ['plate']
}

CCPD_PATH = "yolov5/ccpd.yaml"   

# **Utils**

In [5]:
#cloning the yolov5 repo.
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
%pip install -qr requirements.txt  #dependencies
%cd ..

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17516 (delta 6), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17516/17516), 16.61 MiB | 32.64 MiB/s, done.
Resolving deltas: 100% (11994/11994), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01


In [6]:
os.makedirs(os.path.dirname(CCPD_PATH), exist_ok=True)

#write the .yaml file in /kaggle/working/yolov5/ccpd.yaml
with open(CCPD_PATH, 'w') as f:
    yaml.dump(CONTENT, f, sort_keys=False)

print(f"file added in: {os.getcwd()}/{CCPD_PATH}")

file added in: /kaggle/working/yolov5/ccpd.yaml


In [17]:
#extracting the metadata from each img in this format (image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number)
def decode_plate(s):
    "this method is used for decoding the plate starting from the name of .jpg file"
    idx   = list(map(int, s.split("_")))
    try:
        return PROVINCES[idx[0]] + ALPHA[idx[1]] + "".join(ADS[i] for i in idx[2:])
    except Exception:
        return None


def split_bbox(bbox_str):
    "extracting x1,y1,x2,y2, ex. '283___502_511___591'  →  ['283','502','511','591']"
    tokens = []
    for seg in bbox_str.split("___"):
        tokens.extend(seg.split("_"))
    if len(tokens) == 4 and all(t.isdigit() for t in tokens):
        return map(int, tokens)
    return (None,)*4

In [18]:
def export_yolo(df_split, split_name, img_w, img_h):
    """
    This method is used for creating the dataset for yolov5.
    Input: df_split (dataframe created by preovious method), split_name (train, val or test)
    For each img we need to create .txt file, which contains:
    CLASS_ID, x_center (normalized), y_center (normalized), width, height.
    The file is created on the directory "/kaggle/working/ccpd_yolo_dataset" (OUT_BASE) 
    """
    img_dir = os.path.join(OUT_BASE, "images", split_name)
    lbl_dir = os.path.join(OUT_BASE, "labels", split_name)
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)

    for _, row in df_split.iterrows():
        try:
            x_center = (row["x1_bbox"] + row["x2_bbox"]) / 2 / img_w
            y_center = (row["y1_bbox"] + row["y2_bbox"]) / 2 / img_h
            width = (row["x2_bbox"] - row["x1_bbox"]) / img_w
            height = (row["y2_bbox"] - row["y1_bbox"]) / img_h

            yolo_line = f"{CLASS_ID} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

            base_name = os.path.basename(row["image_path"])
            name_no_ext = os.path.splitext(base_name)[0]

            dst_img_path = os.path.join(img_dir, base_name)
            shutil.copy2(row["image_path"], dst_img_path)

            #label YOLO
            label_path = os.path.join(lbl_dir, f"{name_no_ext}.txt")
            with open(label_path, "w") as f:
                f.write(yolo_line)

        except Exception as e:
            print(f"Error {row['image_path']}: {e}")

    print(f"{split_name.upper()} completed  {len(df_split)} examples")

## **Data**

In [3]:
#download the training datatset, and test dataset (dim(train) = 50k, dim(test) = 8k).
!gdown --folder https://drive.google.com/drive/folders/1YjnqKF3GV8BL-3ab88d5rzlx-skNSmxy -O datasets

Retrieving folder contents
Processing file 1At2itl6TyCoflqlgmAaAvy2J7r6LlxLi ccpd_test.tar
Processing file 14s-xOvv20PXJL_PxujiUogxLw57UjEul ccpd_train.tar
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1At2itl6TyCoflqlgmAaAvy2J7r6LlxLi
From (redirected): https://drive.google.com/uc?id=1At2itl6TyCoflqlgmAaAvy2J7r6LlxLi&confirm=t&uuid=347f8bb6-d2d0-4cb5-b72d-f79d1ecb0017
To: /kaggle/working/datasets/ccpd_test.tar
100%|█████████████████████████████████████████| 557M/557M [00:02<00:00, 192MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=14s-xOvv20PXJL_PxujiUogxLw57UjEul
From (redirected): https://drive.google.com/uc?id=14s-xOvv20PXJL_PxujiUogxLw57UjEul&confirm=t&uuid=de577b69-6cac-4708-9ebf-4362db68e874
To: /kaggle/working/datasets/ccpd_train.tar
100%|███████████████████████████████████████| 3.76G/3.76G [00:36<00:00, 103MB/s]
Download completed


In [4]:
# extracting the .tar archive.
def extract_tar_archive(archive_path, destination_path):

    print(f"Extracting the tar archive in:{archive_path}")
    with tarfile.open(archive_path, "r") as tar:
        tar.extractall(path=destination_path)
        
    print(f"Archive extracted in: {destination_path}")

#delete the .tar archive which now is useless.
def delete_tar_archive(path_tar_archive):
    
    if os.path.exists(path_tar_archive):
        shutil.rmtree(path_tar_archive)
        print(f"Folder eliminated: {path_tar_archive}")
    else:
        print(f"Folder not found: {path_tar_archive}")
        
archive_path_train = "/kaggle/working/datasets/ccpd_train.tar"
archive_path_test = "/kaggle/working/datasets/ccpd_test.tar"
extract_path = "/kaggle/working/"
folder_path = "/kaggle/working/ccpd_subset_base/train"

#when extracting the files, is important to eliminate the .tar archive which now occupy /kaggle/working space.
extract_tar_archive(archive_path_train, extract_path)
extract_tar_archive(archive_path_test, extract_path)
delete_tar_archive("/kaggle/working/datasets/")

Extracting the tar archive in:/kaggle/working/datasets/ccpd_train.tar
Archive extracted in: /kaggle/working/
Extracting the tar archive in:/kaggle/working/datasets/ccpd_test.tar
Archive extracted in: /kaggle/working/
Folder eliminated: /kaggle/working/datasets/


# **Training**

In [19]:
folder = "/kaggle/working/ccpd_subset_base/train"
rows   = []

for fname in os.listdir(folder):
    if not fname.endswith(".jpg"): continue

    parts = fname[:-4].split("-")           
    if len(parts) < 6:
        continue #the ccpd file name is wrong           

    x1,y1,x2,y2 = split_bbox(parts[2])      
    plate = decode_plate(parts[4])    

    rows.append({
        "image_path": os.path.join(folder, fname),
        "x1_bbox": x1, "y1_bbox": y1,
        "x2_bbox": x2, "y2_bbox": y2,
        "plate_number": plate
    })

df = pd.DataFrame(rows)
print("Rows number:", len(df))         
print("Columns numner:", df.shape[1])
print("Shape:", df.shape)
df.head()


Rows number: 50000
Columns numner: 6
Shape: (50000, 6)


,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number
0,/kaggle/working/ccpd_subset_base/train/0252274...,213,480,438,598,皖AZ339F
1,/kaggle/working/ccpd_subset_base/train/0216235...,236,512,527,602,皖RWZ900
2,/kaggle/working/ccpd_subset_base/train/0320581...,220,466,564,572,鄂A32QA6
3,/kaggle/working/ccpd_subset_base/train/0434051...,182,401,532,536,皖APF418
4,/kaggle/working/ccpd_subset_base/train/0314583...,190,473,512,578,浙ARD851


In [20]:
#we split the dataset in 80/20 for training phase.
df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

print(f"Train set: {len(df_train)} img")
print(f"Val set:   {len(df_val)} img")

Train set: 40000 img
Val set:   10000 img


In [21]:
export_yolo(df_train, "train", IMG_W, IMG_H)
export_yolo(df_val, "val", IMG_W, IMG_H)

TRAIN completed  40000 examples
VAL completed  10000 examples


In [23]:
# FASE 1: Iperparametri per il Warm-up (10 epoche)
my_hyp_warmup = {
    # General
    'lr0': 0.01,              # Initial learning-rate
    'lrf': 0.1,               # Final OneCycleLR learning-rate (lr0 * lrf)
    'momentum': 0.937,        # SGD momentum/Adam beta1
    'weight_decay': 0.0005,   # Optimizer weight decay

    # Warm-up
    'warmup_epochs': 3.0,     # Warmup epochs 
    'warmup_momentum': 0.8,   # Warmup initial momentum
    'warmup_bias_lr': 0.1,    # Warmup initial bias lr

    # Loss balance & IoU
    'box': 0.05,              # Box loss gain
    'cls': 0.3,               # Cls loss gain (scale with pixels)
    'cls_pw': 1.0,            # Cls BCELoss positive_weight
    'obj': 1.0,               # Obj loss gain (scale with pixels)
    'obj_pw': 1.0,            # Obj BCELoss positive_weight
    'iou_t': 0.30,            # IoU training threshold
    'anchor_t': 4.0,          # Anchor-multiple threshold
    'fl_gamma': 0.0,          # Focal loss gamma 

    # Augmentation - color / geometry
    'hsv_h': 0.015,           # Image HSV-Hue augmentation (fraction)
    'hsv_s': 0.7,             # Image HSV-Saturation augmentation (fraction)
    'hsv_v': 0.4,             # Image HSV-Value augmentation (fraction)
    'degrees': 20.0,          # Image rotation (+/- deg)
    'translate': 0.2,         # Image translation (+/- fraction)
    'scale': 0.9,             # Image scale (+/- gain)
    'shear': 20.0,            # Image shear (+/- deg)
    'perspective': 0.001,     # Image perspective (+/- fraction), range 0-0.001

    # Augmentation - flip & mix
    'flipud': 0.0,            # Image flip up-down (probability)
    'fliplr': 0.5,            # Image flip left-right (probability)
    'mosaic': 0.0,            # <<< Image mosaic (probability) - DISATTIVATO
    'mixup': 0.0,             # <<< Image mixup (probability) - DISATTIVATO
    'copy_paste': 0.0         # <<< Segment copy-paste (probability) - DISATTIVATO
}


hyp_path_warmup = "yolov5/data/hyps/my_ccpd_warmup.yaml"
os.makedirs(os.path.dirname(hyp_path_warmup), exist_ok=True)
with open(hyp_path_warmup, 'w') as f:
    yaml.dump(my_hyp_warmup, f, sort_keys=False)

print(f"File di iperparametri per la Fase 1 salvato in: {hyp_path_warmup}")

File di iperparametri per la Fase 1 salvato in: yolov5/data/hyps/my_ccpd_warmup.yaml


In [24]:
#training phase A
!wandb disabled
!python -W ignore yolov5/train.py \
  --weights yolov5s.pt \
  --data yolov5/ccpd.yaml \
  --hyp yolov5/data/hyps/my_ccpd_warmup.yaml \
  --batch -1 \
  --epochs 10 \
  --img 416 \
  --name ccpd_warmup \
  --cache

W&B disabled.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-20 13:36:50.698622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753018610.899175    1648 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753018610.961877    1648 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=yolov5/ccpd.yaml, hyp=yolov5/data/hyps/my_ccpd_warmup.yaml, epochs=10, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/d

In [25]:
# FASE 2: 
my_hyp_final = {
    # General
    'lr0': 0.01,              # Initial learning-rate
    'lrf': 0.1,               # Final OneCycleLR learning-rate (lr0 * lrf)
    'momentum': 0.937,        # SGD momentum/Adam beta1
    'weight_decay': 0.0005,   # Optimizer weight decay

    # Warm-up
    'warmup_epochs': 3.0,     # Warmup epochs (fractions ok)
    'warmup_momentum': 0.8,   # Warmup initial momentum
    'warmup_bias_lr': 0.1,    # Warmup initial bias lr

    # Loss balance & IoU
    'box': 0.05,              # Box loss gain
    'cls': 0.3,               # Cls loss gain (scale with pixels)
    'cls_pw': 1.0,            # Cls BCELoss positive_weight
    'obj': 1.0,               # Obj loss gain (scale with pixels)
    'obj_pw': 1.0,            # Obj BCELoss positive_weight
    'iou_t': 0.30,            # IoU training threshold        
    'anchor_t': 4.0,          # Anchor-multiple threshold
    'fl_gamma': 0.0,          # Focal loss gamma 

    # Augmentation - color (per DB, Weather)
    'hsv_h': 0.015,           # Tonalità
    'hsv_s': 0.7,             # Saturazione (per colori sbiaditi)
    'hsv_v': 0.4,             # Luminosità (per scene troppo scure/chiare)

    # Augmentation - geometry (per Rotate, Tilt, FN)
    'degrees': 20.0,          # Aumentato per CCPD-Rotate e Tilt
    'translate': 0.2,         # Variazione della posizione
    'scale': 0.9,             # Range di scala ampio per CCPD-FN (targhe vicine/lontane)
    'shear': 20.0,            # Aumentato per CCPD-Tilt
    'perspective': 0.001,     # Per le inclinazioni di CCPD-Tilt

    # Augmentation - flip & mix (per Challenge, Weather, FN)
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,            # <<< Per FN, Tilt, Rotate e Challenge
    'mixup': 0.2,             # <<< Aumentato per DB e Weather
    'copy_paste': 0.1
}

hyp_path_final = "yolov5/data/hyps/my_ccpd_final.yaml"
os.makedirs(os.path.dirname(hyp_path_final), exist_ok=True)
with open(hyp_path_final, 'w') as f:
    yaml.dump(my_hyp_final, f, sort_keys=False)

print(f"File di iperparametri per la Fase 2 salvato in: {hyp_path_final}")

File di iperparametri per la Fase 2 salvato in: yolov5/data/hyps/my_ccpd_final.yaml


In [26]:
!wandb disabled
!python -W ignore yolov5/train.py \
  --weights yolov5/runs/train/ccpd_warmup/weights/last.pt \
  --data yolov5/ccpd.yaml \
  --hyp yolov5/data/hyps/my_ccpd_final.yaml \
  --batch -1 \
  --epochs 40 \
  --img 416 \
  --name ccpd_final \
  --cache

W&B disabled.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-20 14:27:13.714706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753021633.737754    1765 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753021633.744559    1765 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5/runs/train/ccpd_warmup/weights/last.pt, cfg=, data=yolov5/ccpd.yaml, hyp=yolov5/data/hyps/my_ccpd_final.yaml, epochs=40, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolv

# **Test**

In [27]:
folder = "/kaggle/working/ccpd_test"
rows = []

for root, _, files in os.walk(folder):
    for fname in files:
        if not fname.endswith(".jpg"):
            continue

        parts = fname[:-4].split("-")
        if len(parts) < 6:
            continue

        x1, y1, x2, y2 = split_bbox(parts[2])
        plate = decode_plate(parts[4])
        full_path = os.path.join(root, fname)

        rows.append({
            "image_path": full_path,
            "x1_bbox": x1,
            "y1_bbox": y1,
            "x2_bbox": x2,
            "y2_bbox": y2,
            "plate_number": plate
        })

df = pd.DataFrame(rows)
print(f"Dataset created: {len(df)} rows")
df.head()

Dataset created: 8000 rows


,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number
0,/kaggle/working/ccpd_test/tilt/0741-32_42-174_...,174,333,414,591,皖NAK326
1,/kaggle/working/ccpd_test/tilt/0545-14_31-198_...,198,449,492,604,晋E68B13
2,/kaggle/working/ccpd_test/tilt/0508-30_30-280_...,280,415,474,634,皖AQ6807
3,/kaggle/working/ccpd_test/tilt/0305-16_44-226_...,226,472,437,593,皖ASH205
4,/kaggle/working/ccpd_test/tilt/0743-29_14-424_...,424,330,685,568,皖AL180E


In [28]:
#export_yolo now consider each subfolder for creating the dataset
def export_yolo(df_split, split_name, img_w, img_h):
    base_img_dir = os.path.join(OUT_BASE, "images", split_name)
    base_lbl_dir = os.path.join(OUT_BASE, "labels", split_name)

    count = 0

    for _, row in df_split.iterrows():
        try:
            if None in (row["x1_bbox"], row["y1_bbox"], row["x2_bbox"], row["y2_bbox"]):
                continue
       
            x_center = (row["x1_bbox"] + row["x2_bbox"]) / 2 / img_w
            y_center = (row["y1_bbox"] + row["y2_bbox"]) / 2 / img_h
            width    = (row["x2_bbox"] - row["x1_bbox"]) / img_w
            height   = (row["y2_bbox"] - row["y1_bbox"]) / img_h

            yolo_line = f"{CLASS_ID} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

            img_path = row["image_path"]
            base_name = os.path.basename(img_path)
            name_no_ext = os.path.splitext(base_name)[0]

            rel_subfolder = os.path.basename(os.path.dirname(img_path))

            img_dir = os.path.join(base_img_dir, rel_subfolder)
            lbl_dir = os.path.join(base_lbl_dir, rel_subfolder)
            os.makedirs(img_dir, exist_ok=True)
            os.makedirs(lbl_dir, exist_ok=True)

            dst_img_path = os.path.join(img_dir, base_name)
            shutil.copy2(img_path, dst_img_path)

            label_path = os.path.join(lbl_dir, f"{name_no_ext}.txt")
            with open(label_path, "w") as f:
                f.write(yolo_line)

            count += 1

        except Exception as e:
            print(f"Error {row['image_path']}: {e}")

    print(f"{split_name.upper()} compleated: {count} examples saved.")
    
export_yolo(df, "test", IMG_W, IMG_H)

TEST compleated: 8000 examples saved.


In [29]:
base_dir = "/kaggle/working/ccpd_yolo_dataset"
img_root = os.path.join(base_dir, "images", "test")
lbl_root = os.path.join(base_dir, "labels", "test")
template_yaml_path = "/kaggle/working/yolov5/ccpd_temp.yaml"
weights_path = "/kaggle/working/yolov5/runs/train/ccpd_final/weights/best.pt"

# All subdir (tilt, blur, ..)
subdirs = [d for d in os.listdir(img_root) if os.path.isdir(os.path.join(img_root, d))]

for sub in subdirs:
    img_dir = os.path.join("images/test", sub)  
    lbl_dir = os.path.join("labels/test", sub)

    yaml_content = f"""\
                    path: {base_dir}
                    train: {img_dir}  # not used
                    val: {img_dir}
                    nc: 1
                    names: ['plate']
                    """
    with open(template_yaml_path, "w") as f:
        f.write(yaml_content)

    print(f" Subset: {sub}")
    !python /kaggle/working/yolov5/val.py \
    --weights "{weights_path}" \
    --data    "{template_yaml_path}" \
    --imgsz   640 \
    --task    val  \
    --iou-thres  0.7 \
    --verbose \
    --save-txt --save-conf \
    --name    test_{sub} \
    --project /kaggle/working/yolov5/runs/test \
    --exist-ok

 Subset: tilt
val: data=/kaggle/working/yolov5/ccpd_temp.yaml, weights=['/kaggle/working/yolov5/runs/train/ccpd_final/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=/kaggle/working/yolov5/runs/test, name=test_tilt, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /kaggle/working/ccpd_yolo_dataset/labels/test/tilt.cache... 1000 i
                 Class     Images  Instances          P          R      mAP50   
                   all       1000       1000      0.988       0.99      0.995      0.654
Speed: 0.2ms pre-process, 2.4ms inference, 1.6ms NMS per image at shape (32, 3, 640, 640)
/usr/local/lib/pyth

In [ ]:
#!zip -r ccpd_results.zip /kaggle/working/yolov5/runs/test

## **METRIC COMPUTATION**

In [31]:
base_dir  = "/kaggle/working/ccpd_yolo_dataset"
runs_root = "/kaggle/working/yolov5/runs/test"

subdirs = [
    d for d in os.listdir(os.path.join(base_dir, "images", "test"))
    if os.path.isdir(os.path.join(base_dir, "images", "test", d))
]


def yolo_to_xyxy(xc, yc, w, h):
    return [xc - w/2, yc - h/2, xc + w/2, yc + h/2]

def load_boxes(txt_path):
    if not txt_path.exists():
        return torch.empty((0, 4))
    boxes = []
    for line in txt_path.read_text().strip().splitlines():
        _, xc, yc, w, h, *rest = map(float, line.split())
        boxes.append(yolo_to_xyxy(xc, yc, w, h))
    return torch.tensor(boxes) if boxes else torch.empty((0, 4))

def metrics_one_subset(sub, thr=0.7):
    pred_dir = Path(runs_root) / f"test_{sub}" / "labels"
    gt_dir   = Path(base_dir)   / "labels" / "test" / sub

    tp, fp, total_gt, correct_img = 0, 0, 0, 0   

    for gt_file in gt_dir.glob("*.txt"):
        gt_boxes   = load_boxes(gt_file)               
        pred_boxes = load_boxes(pred_dir / gt_file.name)
        total_gt  += len(gt_boxes)                     

        # accuracy
        if len(gt_boxes) and len(pred_boxes):
            if (box_iou(gt_boxes, pred_boxes).max() >= thr):
                correct_img += 1
                
        # TP / FP box 
        if len(gt_boxes) and len(pred_boxes):
            ious = box_iou(gt_boxes, pred_boxes)
            tp  += (ious.max(dim=1).values > thr).sum().item()
            fp  += (ious.max(dim=0).values <= thr).sum().item()
        elif len(pred_boxes):  
            fp += len(pred_boxes)

    fn = total_gt - tp
    recall    = tp / total_gt if total_gt else 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    accuracy  = correct_img / total_gt if total_gt else 0.0 

    return {
        "subset": sub,
        "GT": total_gt,
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "Recall_IoU>0.7":    round(recall,    4),
        "Precision_IoU>0.7": round(precision, 4),
        "Accuracy_img_IoU>0.7": round(accuracy, 4)
    }

results = [metrics_one_subset(sub) for sub in subdirs]
df = pd.DataFrame(results)

df = df[["subset", "GT", "TP", "FP", "FN",
         "Recall_IoU>0.7", "Precision_IoU>0.7", "Accuracy_img_IoU>0.7"]]

print(df.to_string(index=False))
df.to_csv("/kaggle/working/iou0.7_metrics.csv", index=False)

   subset   GT  TP    FP  FN  Recall_IoU>0.7  Precision_IoU>0.7  Accuracy_img_IoU>0.7
     tilt 1000 956  5625  44           0.956             0.1453                 0.956
challenge 1000 980 10633  20           0.980             0.0844                 0.980
  weather 1000 997  7507   3           0.997             0.1172                 0.997
       db 1000 926 11550  74           0.926             0.0742                 0.926
       fn 1000 930 11253  70           0.930             0.0763                 0.930
     base 1000 996  8220   4           0.996             0.1081                 0.996
   rotate 1000 986  4738  14           0.986             0.1723                 0.986
     blur 1000 949 10746  51           0.949             0.0811                 0.949


In [32]:
base_dir      = "/kaggle/working/ccpd_yolo_dataset"
img_dir_rel   = "images/test"                 
template_yaml = "/kaggle/working/yolov5/ccpd.yaml"
batch_size    = 5 # the paper use batch size 5 for fps computation

yaml_text = f"""\
path: {base_dir}
train: {img_dir_rel}  # not used
val:   {img_dir_rel}
nc: 1
names: ['plate']
"""
with open(template_yaml, "w") as f:
    f.write(yaml_text)

def count_imgs(root):                
    exts = {".jpg", ".jpeg"}
    return sum(1 for p in Path(root).rglob('*') if p.suffix.lower() in exts)

n_imgs = count_imgs(Path(base_dir) / img_dir_rel)
if n_imgs == 0:
    raise RuntimeError("Error: No img found")

print(f"Starting clock for {n_imgs} imgs (batch={batch_size})")

t0 = time.perf_counter() #clock starting

cmd = [
    sys.executable, "-W", "ignore",                  
    "/kaggle/working/yolov5/val.py",
    "--weights", weights_path,
    "--data",    template_yaml,
    "--imgsz",   "640",
    "--batch",   str(batch_size),
    "--task",    "val",
    "--iou-thres","0.7",
    "--conf-thres","0.001",
    "--name",    "ccpd_global",
    "--project", "/kaggle/working/yolov5/runs/test",
    "--exist-ok"
]
result = subprocess.run(cmd, capture_output=True, text=True)
t1 = time.perf_counter() #stop counter

#
elapsed = t1 - t0            # total seconds
fps     = n_imgs / elapsed   # fps computation

print(f"Total img   : {n_imgs}")
print("Formula              : FPS = num_images / elapsed_seconds")
print(f"Computation              : {n_imgs} / {elapsed:.2f} s  =  {fps:,.1f} FPS")

Starting clock for 8000 imgs (batch=5)
Total img   : 8000
Formula              : FPS = num_images / elapsed_seconds
Computation              : 8000 / 84.85 s  =  94.3 FPS
